<a href="https://colab.research.google.com/github/QaziSaim/DeepLearning--DeepNeuralNetwork-/blob/main/LSTM_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sahimkazi","key":"1f706c6d27536904b27f4957ba8b495b"}'}

In [7]:
!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory

-rw-r--r-- 1 root root 65 Sep 12 05:41 kaggle.json
/content


In [8]:
!chmod 600 ~/.kaggle/kaggle.json


In [9]:
!kaggle datasets list

ref                                                        title                                                  size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -----------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
yashdevladdha/uber-ride-analytics-dashboard                Uber Data Analytics Dashboard                      17324552  2025-08-08 11:13:42.920000          46942       1038  1.0              
mdsultanulislamovi/student-stress-monitoring-datasets      Student Stress Monitoring Datasets                    24336  2025-07-24 16:30:01.617000          26023        432  1.0              
msnbehdani/mock-dataset-of-second-hand-car-sales           Car Sales Dataset: Model, Features, and Pricing      501188  2025-08-20 17:47:58.207000           3275         55  1.0              
zubairamuti/shopping-behaviours-dataset 

In [10]:
#!/bin/bash
!kaggle datasets download hgultekin/bbcnewsarchive

Dataset URL: https://www.kaggle.com/datasets/hgultekin/bbcnewsarchive
License(s): copyright-authors
  0% 0.00/1.81M [00:00<?, ?B/s]
100% 1.81M/1.81M [00:00<00:00, 100MB/s]


In [11]:
!unzip /content/bbcnewsarchive.zip

Archive:  /content/bbcnewsarchive.zip
  inflating: bbc-news-data.csv       


In [13]:
import pandas as pd

In [14]:
df = pd.read_csv('/content/bbc-news-data.csv',sep='\t')

In [15]:
df.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [16]:
df['category'].value_counts()

,count
category,
sport,511
business,510
politics,417
tech,401
entertainment,386


In [17]:
df.dropna(inplace=True)

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
df.isnull().sum()

,0
category,0
filename,0
title,0
content,0


In [20]:
df.shape

(2225, 4)

In [21]:
df.size

8900

In [22]:
df.ndim

2

In [23]:
import tensorflow
from tensorflow.keras.layers import SimpleRNN,Dense,Embedding,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
def data_clean(text):
  text = text.lower()
  text = text.replace('.','. ')
  tokens = [word for word in text.split() if not word in stop_words]
  text = " ".join(tokens)
  text = text.strip()
  return (text)

In [26]:
df['title'] = df['title'].apply(lambda x:data_clean(x))

In [27]:
df.head()

,category,filename,title,content
0,business,001.txt,ad sales boost time warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,dollar gains greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,high fuel prices hit ba's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,pernod takeover talk lifts domecq,Shares in UK drinks and food firm Allied Dome...


In [28]:
max(df['title'].str.split(' ').str.len())

7

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train,X_test,y_train,y_test = train_test_split(df['title'],df['category'],test_size=0.2,random_state=42)

In [31]:
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

In [32]:
y_train.head()

,category
0,sport
1,tech
2,sport
3,tech
4,entertainment


In [33]:
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values

In [34]:
y_train

array([[False, False, False,  True, False],
       [False, False, False, False,  True],
       [False, False, False,  True, False],
       ...,
       [False, False,  True, False, False],
       [False, False,  True, False, False],
       [False,  True, False, False, False]])

In [35]:
# convert to integer # tokenize # padd
def tokenize_and_pad(input_text,max_len,tok):
  input_text = tok.texts_to_sequences(input_text) #
  print(input_text) # word converted to integer
  input_text = pad_sequences(input_text, maxlen = max_len, padding='post')
  print(input_text) # words converted to integer pluse padding
  return input_text


In [36]:
text_tok = Tokenizer()
text_tok.fit_on_texts(X_train)

In [37]:
X_train_text=tokenize_and_pad(X_train,7,text_tok)

[[495, 779, 496, 358, 1390], [1391, 196, 497, 74], [780, 498, 17], [1392, 2, 781, 499, 359], [500, 93, 782, 783], [1, 156, 197, 8, 43, 141], [1393, 1394, 784, 501, 360], [1395, 1396, 10, 1397, 94], [502, 1398, 198, 503, 157], [142, 49, 199, 785, 50], [5, 266, 1399, 1400, 200], [1401, 50, 1402, 1403, 1404], [1405, 786, 8, 1406, 74], [1407, 1408, 40, 95], [1409, 267, 504, 505, 1410], [1411, 268, 96, 1412], [3, 506, 361, 787, 33], [1413, 10, 201, 362], [507, 158, 508, 363, 267], [269, 1414, 788, 1415], [75, 509, 202, 159, 789], [790, 364, 1416, 791], [510, 365, 160], [366, 511, 1417, 270], [1, 268, 792, 512], [367, 793, 794], [1418, 368, 57, 1419], [1420, 795, 796, 513, 271], [514, 112, 369, 797, 76], [1421, 1422, 203, 515], [1423, 1424, 1], [1, 798, 516, 1425, 1426], [517, 272, 1427, 367], [204, 518, 1428, 799], [800, 519, 801, 273], [44, 45, 274, 1429], [51, 1430, 275, 520, 1431], [1432, 802, 205], [276, 1433, 206, 1434, 1435], [277, 278, 803, 77, 804], [113, 52, 1436, 805], [156, 197, 

In [38]:
X_train

,title
0,cole refuses blame van persie
1,slimmer playstation triple sales
2,bellamy fined row
3,finding new homes old phones
4,sundance honour foreign films
...,...
1775,robinson answers critics
1776,mallon wades ne vote battle
1777,lib dems' new election pr chief
1778,tories reject rethink axed mp
